In [1]:
import scanpy as sc
import tempfile
import subprocess
import os

In [4]:
data_dir = '../data/'

In [5]:
adata = sc.read(data_dir+'processed/04seurat_integrated_noprior.h5ad')

In [6]:
sc.pp.neighbors(adata)
sc.tl.louvain(adata, resolution=0.5, key_added='louvain_post')

         Falling back to preprocessing with `sc.pp.pca` and default params.


## Trying code

In [9]:
group1 = 'louvain'
group2 = 'louvain_post'
group_name = group1

In [29]:
clusters = {label:[] for label in adata.obs[group_name].unique()}
for i, label in enumerate(adata.obs[group_name]):
    clusters[label].append(str(i))

In [36]:
def write_tmp_labels(adata, group_name='louvain'):
    
    clusters = {label:[] for label in adata.obs[group_name].unique()}
    for i, label in enumerate(adata.obs[group_name]):
        clusters[label].append(str(i))
    
    f = tempfile.NamedTemporaryFile(delete=False)
    f.write(str.encode('\n'.join([' '.join(c) for c in clusters.values()])))
    f.close()
    
    return f.name

In [37]:
group1_file = write_tmp_labels(adata, group1)
group2_file = write_tmp_labels(adata, group2)

In [44]:
os.path.abspath("../../../mutual3/")

'/home/m/muellermi/Dokumente/Master2/Masterpraktikum/mutual3'

In [50]:
nmi_dir = "../../../mutual3/"
nmi_call = subprocess.Popen(
    [nmi_dir+"mutual", group1_file, group2_file], 
    stdout=subprocess.PIPE, 
    stderr=subprocess.STDOUT)
stdout, stderr = nmi_call.communicate()
print(stderr)
nmi_out = stdout.decode()
nmi_out.strip().split('\t')[1]

None


'0.574931'

## Finished Function

In [15]:
def write_tmp_labels(adata, group_name):
    label_map = {}
    i = 1
    for label in adata.obs[group_name].unique():
        label_map[label] = i
        i += 1
    labels = '\n'.join([str(label_map[name]) for name in adata.obs[group_name]])
    with tempfile.NamedTemporaryFile(delete=False) as f:
        f.write(str.encode(labels))
        filename = f.name
    return filename

In [19]:
def nmi(adata, group1, group2, onmi_dir="../../Overlapping-NMI/"):
    group1_file = write_tmp_labels(adata, group1)
    group2_file = write_tmp_labels(adata, group2)
    
    nmi_call = subprocess.Popen(
        [onmi_dir+"onmi", group1_file, group2_file], 
        stdout=subprocess.PIPE, 
        stderr=subprocess.STDOUT)
    
    stdout, stderr = nmi_call.communicate()
    print(stderr)
    nmi_out = stdout.decode()
    print(nmi_out)
    
    nmi_split = [x.strip().split('\t') for x in nmi_out.split('\n')]
    nmi_max = nmi_split[0][1]
    
    # remove temporary files
    os.remove(group1_file)
    os.remove(group2_file)
    return nmi_max

In [20]:
nmi(adata, group1='louvain', group2='louvain_post', onmi_dir="../../Overlapping-NMI/")

None
NMI<Max>:	0.998732
Other measures:
  lfkNMI:	0.998732
  NMI<Sum>:	0.998732



'0.998732'